In [8]:
from config.loader import load_config
import numpy as np

# 加载配置
config = load_config()

# 访问配置项
db_host = config['robot_ip']
fx = config['DEPTH_INTR']['fx']
translation_vector = config['rotation_matrix']

print(f"robot ip is: {db_host}")
print('try to find type and value of fx', '\n', type(fx), fx)
print('try to find type and value of translation_vector', '\n', type(translation_vector), translation_vector, type(translation_vector[1]), translation_vector[1])
print(len(translation_vector))
print(np.array(translation_vector))

robot ip is: 192.168.1.19
try to find type and value of fx 
 <class 'float'> 646.495
try to find type and value of translation_vector 
 <class 'list'> [[-0.05578176, -0.99668141, -0.0592837], [0.99840848, -0.05518776, -0.01161139], [0.00830112, -0.05983705, 0.99817364]] <class 'list'> [0.99840848, -0.05518776, -0.01161139]
3
[[-0.05578176 -0.99668141 -0.0592837 ]
 [ 0.99840848 -0.05518776 -0.01161139]
 [ 0.00830112 -0.05983705  0.99817364]]


In [1]:
from Robotic_Arm.rm_robot_interface import *

# 实例化RoboticArm类
arm = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)

# 创建机械臂连接，打印连接id
handle = arm.rm_create_robot_arm("192.168.110.119", 8080)
print(handle.id)

a, b = arm.rm_get_current_arm_state()
print(b)
pose = b['pose']
mat = arm.rm_algo_pos2matrix(pose)
print(mat)
out = arm.rm_algo_matrix2pos(mat)
print(out, type(out))


arm.rm_delete_robot_arm()

current c api version:  1.0.6
1
{'joint': [-0.0010000000474974513, 0.0, 0.0, 0.0020000000949949026, -0.0010000000474974513, 0.0010000000474974513], 'pose': [3e-06, 0.0, 0.8677, 0.0, 0.0, 3.141], 'err': {'err_len': 1, 'err': ['0x0000']}}
[3.000000106112566e-06, 0.0, 0.8676999807357788, 0.0, 0.0, 3.1410000324249268] <class 'list'>


0

In [1]:
import pyrealsense2 as rs
import numpy as np
import cv2
from func01 import pixel_to_camera_coord
from Robotic_Arm.rm_robot_interface import *
import time

# 实例化RoboticArm类
arm = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)

# 创建机械臂连接，打印连接id
handle = arm.rm_create_robot_arm("192.168.110.118", 8080)
# print(handle.id)
init_pose = [-0.20807, 0.000295, 0.488525, 3.142, 0.326, 0.001]
init1 = [-0.20807, 0.000295, 0.488525, 3.142, 0, 0]

# 初始化管道
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)  # RGB流
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)  # 深度流

# config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 6)  # RGB流
# config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 6)  # 深度流

pipeline.start(config)
align = rs.align(rs.stream.color)

# 鼠标回调函数
depth_scale = None
point_selected = False

def mouse_callback(event, x, y, flags, param):
    global depth_scale, point_selected, arm, pose_collect
    if event == cv2.EVENT_LBUTTONDOWN:
        # 获取深度帧
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        depth_frame = aligned_frames.get_depth_frame()
        
        # 获取深度比例因子
        if depth_scale is None:
            depth_sensor = pipeline.get_active_profile().get_device().first_depth_sensor()
            depth_scale = depth_sensor.get_depth_scale()
        
        # 获取深度值（米）
        depth_m = depth_frame.get_distance(x, y)
        # print('depth scale', depth_scale)
        # depth_m = depth_frame.get_distance(x, y) * depth_scale
        print(f"深度值（米）: {depth_m:.3f}")
        
        # 获取RGB内参
        color_profile = pipeline.get_active_profile().get_stream(rs.stream.color).as_video_stream_profile().get_intrinsics()
        fx = color_profile.fx
        fy = color_profile.fy
        cx = color_profile.ppx
        cy = color_profile.ppy
        # 坐标转换, 相机坐标系坐标
        Xc = (x - cx) / fx * depth_m
        Yc = (y - cy) / fy * depth_m
        Zc = depth_m
        point_cam = np.array([Xc, Yc, Zc])


        # 转换成机械臂基坐标系下的坐标
        R_cam_base = np.array(  [[-0.00310044 , 0.99999236 , 0.00238261],
                                [ -0.99997374 ,-0.00311597  ,0.00654238],
                                [ 0.00654975 ,-0.00236226 , 0.99997576]])
        t_cam_base = np.array( [[-0.09674632,0.03282877,0.0294021 ]])
        point_base = R_cam_base @ point_cam + t_cam_base
        point_base = np.array(point_base, dtype=np.float32).reshape(3, 1)
        # print(type(point_base), np.shape(point_base))
        a, b = arm.rm_get_current_arm_state()
        pose = b['pose']
        joint = b['joint']
        # print(pose)
        mat = arm.rm_algo_pos2matrix(pose)
        # print(mat)
        mat = np.array(mat.data, dtype=np.float32).reshape(4, 4)
        rot, vec = mat[:3, :3], mat[:3, -1].reshape(1, 3)
        # print(type(rot), np.shape(rot), type(vec), np.shape(vec))

        point_base = rot.dot(point_base)
        point_base = point_base.reshape(1, 3)
        point_base += vec
        
        # print(f"像素坐标： ({x}, {y})")
        # print(f"相机坐标系坐标: ({Xc:.3f}, {Yc:.3f}, {Zc:.3f})")
        print(f'机械臂基坐标系下坐标： ({point_base})')
        pose1 = init1.copy()
        for i in range(3):
            pose1[i] = point_base[0][i]
        error_code, new_joint = arm.rm_algo_inverse_kinematics(rm_inverse_kinematics_params_t(joint, pose1, 1))
        if error_code != 0:
            print('fail to do inverse calculation')
        else:
            print('inverse calculation succeed!')
            print(new_joint)
            new_joint = arm.rm_algo_cartesian_tool(new_joint, 0, 0, -0.05)
            print(new_joint)
            print(arm.rm_movej_p(new_joint, 10, 0, 0, 1))
            time.sleep(10)
            print(arm.rm_movej_p(init_pose, 20, 0, 0, 1))
        point_selected = True

# 创建窗口并绑定回调
cv2.namedWindow('RGB & Depth')
cv2.setMouseCallback('RGB & Depth', mouse_callback)

try:
    while True:
        # 获取对齐后的帧
        frames = pipeline.wait_for_frames()
        aligned_frames = align.process(frames)
        color_frame = aligned_frames.get_color_frame()
        depth_frame = aligned_frames.get_depth_frame()
        
        # 转换为numpy数组
        color_image = np.asanyarray(color_frame.get_data())
        depth_image = np.asanyarray(depth_frame.get_data())
        # print('color_image', np.shape(color_image), 'depth_image', np.shape(depth_image))
        
        # 可视化深度图（彩色映射）
        depth_colormap = cv2.applyColorMap(cv2.convertScaleAbs(depth_image, alpha=0.03), cv2.COLORMAP_JET)
        
        # 合并显示RGB和彩色深度图
        # combined_image = np.hstack((color_image, depth_image))
        combined_image = np.hstack((color_image, depth_colormap))
        
        # 显示图像
        cv2.imshow('RGB & Depth', combined_image)
        # cv2.imshow('rbg', color_image)
        # cv2.imshow('Depth', depth_image)
        
        if point_selected:
            # 在图像上标记选中的点
            # cv2.circle(color_image, (x, y), 5, (0, 255, 0), -1)
            cv2.imshow('RGB & Depth', combined_image)
            point_selected = False
        
        if cv2.waitKey(1) & 0xFF == ord('q'):
            arm.rm_delete_robot_arm()
            break
finally:
    pipeline.stop()
    cv2.destroyAllWindows()

current c api version:  1.0.6


In [35]:
a = [-0.43634102 , 0.04662181 ,-0.00998363]
b = [-0.43667188 ,-0.00388653 ,-0.00951245]

out = ((a[0] - b[0])**2 + (a[1] - b[1])**2 + (a[2] - b[2])**2)**0.5
print(out)

0.050511621321113816


In [ ]:
from Robotic_Arm.rm_robot_interface import *

# 实例化RoboticArm类
arm = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)
init = [-0.20807, 0.000295, 0.488525, 3.142, 0.326, 0.001]
pose1 = [-0.5774, 0.0364, 0.1097, 0, 3.14, 0]
pose2 = [-0.519493, -0.031722, 0.078806, -1.808, 1.21, 0.493]
# 创建机械臂连接，打印连接id
handle = arm.rm_create_robot_arm("192.168.110.118", 8080)

a, b = arm.rm_get_current_arm_state()
print(b)
pose = b['pose']
print(pose)

# print(arm.rm_movej_p(init, 20, 0, 0, 1))
# print(arm.rm_movej_p(pose1, 10, 0, 0, 1))
# print(arm.rm_movej_p(pose, 10, 0, 0, 1))
print(arm.rm_movej([0,0,0,0,0,0], 20, 0, 0, 1))

arm.rm_delete_robot_arm()

current c api version:  1.0.6
{'joint': [-0.10300000011920929, 0.9679999947547913, 45.44900131225586, 0.02800000086426735, 114.90599822998047, -0.14100000262260437], 'pose': [-0.208066, 0.0003, 0.488526, 3.142, 0.325, 0.001], 'err': {'err_len': 1, 'err': ['0x0000']}}
[-0.208066, 0.0003, 0.488526, 3.142, 0.325, 0.001]
0


0

: 

In [ ]:
import pyrealsense2 as rs
import numpy as np

# 初始化管道
pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)
config.enable_stream(rs.stream.depth, 640, 480, rs.format.z16, 30)
# config.enable_stream(rs.stream.color, 1280, 720, rs.format.bgr8, 6)
# config.enable_stream(rs.stream.depth, 1280, 720, rs.format.z16, 6)
pipeline.start(config)

# 获取 RGB 相机内参
color_frame = pipeline.wait_for_frames().get_color_frame()
color_intrin = color_frame.profile.as_video_stream_profile().intrinsics
fx, fy, cx, cy = color_intrin.fx, color_intrin.fy, color_intrin.ppx, color_intrin.ppy
dist_coeffs = color_intrin.coeffs  # 畸变系数
dist_coeffs = np.array(dist_coeffs)

# 获取深度相机内参（与 RGB 共享）
# depth_intrin = color_intrin  # D435i 的 RGB 和深度相机内参对齐

depth_frame = pipeline.wait_for_frames().get_depth_frame()
depth_intrin = depth_frame.profile.as_video_stream_profile().intrinsics
fx1, fy1, cx1, cy1 = depth_intrin.fx, depth_intrin.fy, depth_intrin.ppx, depth_intrin.ppy
dist_coeffs1 = depth_intrin.coeffs  # 畸变系数
dist_coeffs1 = np.array(dist_coeffs1)

# 停止管道
pipeline.stop()

K = np.array([
    [fx, 0, cx],
    [0, fy, cy],
    [0, 0, 1]
], dtype=np.float32)

print(color_intrin, '\n', fx, fy, cx, cy, dist_coeffs, '\n', K)
print(depth_intrin, '\n', fx1, fy1, cx1, cy1, dist_coeffs1)

[ 1280x720  p[652.811 356.065]  f[913.225 911.32]  Inverse Brown Conrady [0 0 0 0 0] ] 
 913.2247314453125 911.320068359375 652.81103515625 356.0649719238281 [0. 0. 0. 0. 0.] 
 [[913.22473   0.      652.81104]
 [  0.      911.32007 356.06497]
 [  0.        0.        1.     ]]
[ 1280x720  p[637.515 358.546]  f[637.481 637.481]  Brown Conrady [0 0 0 0 0] ] 
 637.4808959960938 637.4808959960938 637.5151977539062 358.5460205078125 [0. 0. 0. 0. 0.]


In [1]:
# 下面是一个如何注册UDP机械臂实时状态主动上报信息回调函数的示例：
# 在这个示例中，我们定义了一个名为`arm_state_func`的函数，用于处理机械臂实时上报的数据，并将其注册为回调函数。
# `arm_state_func`函数会按照UDP接口设置的周期被调用，该函数接收一个rm_realtime_arm_joint_state_t的对象作为参数
from Robotic_Arm.rm_robot_interface import *
import keyboard
import time
import numpy as np
import cv2
import pyrealsense2 as rs
import os

def arm_state_func(data):
    global pipeline, images
    a = data.joint_status.to_dict()
    frames = pipeline.wait_for_frames()
    color_frame = frames.get_color_frame()
    color_image = np.asanyarray(color_frame.get_data())
    images.append(color_image.copy())
    print("Current arm pose: ", a['joint_position'])


# 初始化为三线程模式
arm = RoboticArm(rm_thread_mode_e.RM_TRIPLE_MODE_E)

# 创建机械臂连接，打印连接id
handle = arm.rm_create_robot_arm("192.168.110.119", 8080)
print(handle.id)

# 设置UDP端口，广播周期500ms，使能，广播端口号8089，力数据坐标系使用传感器坐标系，上报目标IP为"192.168.1.104"
# 自定义上报项均设置关闭，用户可根据实际情况修改这些配置
custom = rm_udp_custom_config_t()
custom.joint_speed = 1
custom.lift_state = 0
custom.expand_state = 0
custom.arm_current_status = 1
config = rm_realtime_push_config_t(50, True, 8089, 0, "192.168.110.55", custom)
print(arm.rm_set_realtime_push(config))
print(arm.rm_get_realtime_push())

pipeline = rs.pipeline()
config = rs.config()
config.enable_stream(rs.stream.color, 640, 480, rs.format.bgr8, 30)  # RGB流
profile = pipeline.start(config)
frames = pipeline.wait_for_frames()
images = []

arm_state_callback = rm_realtime_arm_state_callback_ptr(arm_state_func)
arm.rm_realtime_arm_state_call_back(arm_state_callback)

start_time = time.time()
# 关节运动
ret = arm.rm_movej([0, 30, 60, 0, 90, 0], 30, 0, 0, 1)
print("movej: ", ret)
ret = arm.rm_movej([0, 0, 0, 0, 0, 0], 30, 0, 0, 1)
print("movej: ", ret)

config_end = rm_realtime_push_config_t(50, False, 8089, 0, "192.168.110.55", custom)
print(arm.rm_set_realtime_push(config_end))
print(arm.rm_get_realtime_push())
end_time = time.time()
print("Time taken for the operation: ", end_time - start_time)
print(end_time)
# 删除指定机械臂对象
arm.rm_delete_robot_arm()
# arm.rm_destory()
foler = os.path.join(os.getcwd(), 'images')
if not os.path.exists(foler):
    os.makedirs(foler)
for i, image in enumerate(images):
    image_path = os.path.join(foler, f'image_{i}.jpg')
    cv2.imwrite(image_path, image)
    print(f"Saved {image_path}")

arm.rm_destory()

current c api version:  1.0.6
1
0
(0, {'cycle': 50, 'enable': True, 'port': 8089, 'force_coordinate': 0, 'ip': '192.168.110.55', 'custom_config': {'joint_speed': 1, 'lift_state': 0, 'expand_state': 0, 'hand_state': 0, 'arm_current_status': 1, 'aloha_state': 0}})
Current arm pose:  [0.014999999664723873, 0.03500000014901161, 0.210999995470047, 0.03799999877810478, 0.21400000154972076, 0.0010000000474974513, 0.0]
Current arm pose:  [-0.006000000052154064, 2.5209999084472656, 5.122000217437744, -0.00800000037997961, 7.486999988555908, 0.0010000000474974513, 0.0]
Current arm pose:  [-0.03400000184774399, 7.824999809265137, 15.63700008392334, -0.04800000041723251, 23.347000122070312, -0.0010000000474974513, 0.0]
Current arm pose:  [-0.01899999938905239, 13.458999633789062, 26.958999633789062, -0.03700000047683716, 40.23699951171875, -0.0010000000474974513, 0.0]
Current arm pose:  [-0.004000000189989805, 19.167999267578125, 38.34199905395508, -0.0020000000949949026, 57.513999938964844, 0.0, 

0

In [2]:
print(np.shape(images))

(15, 480, 640, 3)
